In [ ]:
!git clone https://github.com/cherrybritty/DS-ASSIGNMENT-WEEK08.git

fatal: destination path 'DS-ASSIGNMENT-WEEK08' already exists and is not an empty directory.


In [ ]:
import pandas as pd
import numpy as np
import string
import re


In [ ]:
df = pd.read_csv('wine-ratings.csv')

In [ ]:
non_dm_data = df.drop_duplicates().copy()
non_dm_data.dropna(inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)
non_dm_data.loc[:,'city']= non_dm_data['region'].str.split(',').str[-2].str.lower().str.strip()
non_dm_data.loc[:,'country']= non_dm_data['region'].str.split(',').str[-1].str.lower().str.strip()
non_dm_data.groupby(['city', 'country']).size().reset_index(name='count').sort_values(by='count', ascending=False).head(10)



,city,country,count
60,napa valley,california,3893
97,sonoma county,california,2973
8,bordeaux,france,2769
81,rhone,france,2063
108,tuscany,italy,2021
16,central coast,california,1846
19,columbia valley,washington,1551
10,burgundy,france,1411
122,willamette valley,oregon,1260
98,south australia,australia,1097


In [ ]:
non_dm_data.loc[non_dm_data['city'] == 'paris', 'country'] = non_dm_data.loc[non_dm_data['city'] == 'paris', 'country'].fillna('france')

In [ ]:
wine_variety_distribution = non_dm_data['variety'].value_counts(normalize=True) * 100
non_dm_data.loc[:,'wine_variety_distribution'] = non_dm_data['variety'].map(wine_variety_distribution)

print(wine_variety_distribution.round(1))

variety
Red Wine                 72.0
White Wine               23.0
Sparkling & Champagne     2.6
Pink and Rosé             1.5
Collectible               0.7
Green Wine                0.1
Boutique                  0.1
Screw Cap                 0.0
Name: proportion, dtype: float64


In [ ]:
def classify_wine(rating):
    if 85 <= rating <= 89:
        return 'Good'
    elif 90 <= rating <= 94:
        return 'Outstanding'
    elif 95 <= rating <= 100:
        return 'Exceptional'
    return np.nan

non_dm_data.loc[:, 'wine_quality'] = non_dm_data['rating'].apply(classify_wine)


In [ ]:
non_dm_data_2 = non_dm_data.copy()


In [ ]:
non_dm_data_2['cleaned_text'] = non_dm_data_2['notes'].str.lower()


In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_punctuation(cleaned_text):
    punctuations = string.punctuation
    return cleaned_text.translate(str.maketrans('', '', string.punctuation))


In [ ]:
non_dm_data_2.loc[:, 'punctuated_text'] = non_dm_data_2['cleaned_text'].apply(lambda x: remove_punctuation(x))

In [ ]:
import re

def remove_numbers(punctuated_text):
    return re.sub(r'\d+', '', punctuated_text)

In [ ]:
non_dm_data_2.loc[:, 'numbers_cleaned'] = non_dm_data_2['punctuated_text'].apply(lambda x: remove_numbers(x))

In [ ]:
import nltk
from nltk import pos_tag
nltk.download('wordnet')
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

def lemmatize_words(numbers_cleaned, separator=" "):
    lemmatized_text = pos_tag(numbers_cleaned.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in lemmatized_text])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

non_dm_data_2.loc[:, 'lemmatized_text'] = non_dm_data_2['numbers_cleaned'].apply(lambda x: lemmatize_words(x))

non_dm_data_2.loc[:, 'description_token'] = non_dm_data_2['lemmatized_text'].apply (lambda x: " ".join(word for word in x.split() if word not in stop_words))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
non_dm_data_2.drop(columns=['city', 'lemmatized_text', 'punctuated_text', 'numbers_cleaned', 'cleaned_text', 'region'], inplace=True)

In [ ]:
non_dm_data_2 = non_dm_data_2.rename(columns={'country': 'Region'})

In [ ]:
from collections import Counter

def get_top_words(non_dm_data_2, quality, top_words=5):
    text = " ".join(non_dm_data_2[non_dm_data_2["wine_quality"] == quality]["description_token"].astype(str).tolist())
    words = text.split()
    word_counts = Counter(words)
    return word_counts.most_common(top_words)


qualities = ["Good", "Outstanding", "Exceptional"]
for quality in qualities:
    top_words = get_top_words(non_dm_data_2, quality)
    print(f"Top 5 words for {quality} wines:")
    for word, count in top_words:
        print(f" . {word}: {count}")


Top 5 words for Good wines:
 . wine: 3544
 . fruit: 2783
 . flavor: 2133
 . aroma: 1783
 . finish: 1750
Top 5 words for Outstanding wines:
 . wine: 20052
 . fruit: 14836
 . aroma: 9064
 . finish: 8804
 . palate: 8747
Top 5 words for Exceptional wines:
 . wine: 2148
 . fruit: 1282
 . tannin: 730
 . black: 719
 . palate: 705


In [ ]:
top_exceptional_regions = non_dm_data_2[non_dm_data_2['wine_quality'] == 'Exceptional']['Region'].value_counts().head(5)
print(top_exceptional_regions)

Region
france        994
california    804
italy         269
australia     124
spain         108
Name: count, dtype: int64


In [ ]:
Region_diversity = non_dm_data_2.groupby('Region')['variety'].nunique().sort_values(ascending=False)
print(Region_diversity.head(10))

Region
australia       8
italy           7
california      7
france          7
spain           6
south africa    6
oregon          6
portugal        5
other u.s.      5
washington      5
Name: variety, dtype: int64
